In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import os

In [2]:
os.chdir(r'C:\SML_Projects\SML_CVE_type_cwe_predict')

In [3]:
df = pd.read_csv("data/preprocessed/preprocessed_dataset.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102963 entries, 0 to 102962
Data columns (total 29 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   cve_id                      102963 non-null  float64
 1   description                 102963 non-null  float64
 2   cvss_score                  102963 non-null  float64
 3   cwe                         102963 non-null  float64
 4   vendor                      102963 non-null  float64
 5   product                     102963 non-null  float64
 6   publish_date                102963 non-null  float64
 7   type                        102963 non-null  float64
 8   vendor_freq                 102963 non-null  float64
 9   product_freq                102963 non-null  float64
 10  desc_len                    102963 non-null  float64
 11  desc_word_count             102963 non-null  float64
 12  desc_num_count              102963 non-null  float64
 13  desc_upper_rat

In [5]:
df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10296 entries, 0 to 10295
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   cve_id                      10296 non-null  float64
 1   description                 10296 non-null  float64
 2   cvss_score                  10296 non-null  float64
 3   cwe                         10296 non-null  float64
 4   vendor                      10296 non-null  float64
 5   product                     10296 non-null  float64
 6   publish_date                10296 non-null  float64
 7   type                        10296 non-null  float64
 8   vendor_freq                 10296 non-null  float64
 9   product_freq                10296 non-null  float64
 10  desc_len                    10296 non-null  float64
 11  desc_word_count             10296 non-null  float64
 12  desc_num_count              10296 non-null  float64
 13  desc_upper_ratio            102

In [7]:
x = df.drop(['type', 'cvss_score'], axis=1)   
y = df[['type', 'cvss_score']] 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [8]:
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

lr = LogisticRegression(max_iter=1000)
multi_lr = MultiOutputClassifier(lr)

multi_lr.fit(x_train, y_train)

importances = np.mean([np.mean(np.abs(est.coef_), axis=0) for est in multi_lr.estimators_], axis=0)

selected_features = x_train.columns[importances > np.mean(importances)]

multi_lr.fit(x_train[selected_features], y_train)
y_pred = multi_lr.predict(x_test[selected_features])

lr_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
lr_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'],  y_pred[:, 1])

lr_scores_type = cross_val_score(lr, x[selected_features], y['type'], cv=kf, scoring='f1_macro')
lr_scores_cvss_score  = cross_val_score(lr, x[selected_features], y['cvss_score'],  cv=kf, scoring='f1_macro')

print("Logistic Regression Accuracy for 'type':", lr_accuracy_type)
print("Logistic Regression Accuracy for 'cvss_score' :", lr_accuracy_cvss_score)

print("K-Fold mean F1 (type):", lr_scores_type.mean())
print("K-Fold std  F1 (type):", lr_scores_type.std())

print("K-Fold mean F1 (cvss_score):", lr_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", lr_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:, 0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'],  y_pred[:, 1]))

Logistic Regression Accuracy for 'type': 0.7490291262135922
Logistic Regression Accuracy for 'cvss_score' : 0.5859223300970874
K-Fold mean F1 (type): 0.5915255055022438
K-Fold std  F1 (type): 0.0077130697323266615
K-Fold mean F1 (cvss_score): 0.28092874200716555
K-Fold std  F1 (cvss_score): 0.0032352530956103287

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        28
         1.0       0.91      0.80      0.85        50
         2.0       0.81      0.92      0.86       103
         3.0       0.02      0.01      0.01       200
         4.0       0.67      0.83      0.74       834
         5.0       0.90      0.68      0.78        41
         6.0       0.41      0.47      0.44        36
         7.0       0.84      0.61      0.70       155
         8.0       1.00      0.98      0.99       176
         9.0       0.00      0.00      0.00        14
        10.0       0.95      0.95      0.95       423



C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classif

# Decision Tree

In [10]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
multi_dt = MultiOutputClassifier(dt)

multi_dt.fit(x_train, y_train)

importances = np.mean([est.feature_importances_ for est in multi_dt.estimators_], axis=0)
selected_features = x_train.columns[importances > np.mean(importances)]

multi_dt.fit(x_train[selected_features], y_train)
y_pred = multi_dt.predict(x_test[selected_features])

dt_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
dt_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

dt_scores_type = cross_val_score(dt, x[selected_features], y['type'], cv=kf, scoring='f1_macro')
dt_scores_cvss_score  = cross_val_score(dt, x[selected_features], y['cvss_score'], cv=kf, scoring='f1_macro')

print("Decision Tree Accuracy for 'type':", dt_accuracy_type)
print("Decision Tree Accuracy for 'cvss_score' :", dt_accuracy_cvss_score)

print("\nK-Fold mean F1 (type):", dt_scores_type.mean())
print("K-Fold std  F1 (type):", dt_scores_type.std())

print("\nK-Fold mean F1 (cvss_score):", dt_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", dt_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Decision Tree Accuracy for 'type': 0.9004854368932039
Decision Tree Accuracy for 'cvss_score' : 0.5912621359223301

K-Fold mean F1 (type): 0.8406609308704635
K-Fold std  F1 (type): 0.009677579359983248

K-Fold mean F1 (cvss_score): 0.46141262424894536
K-Fold std  F1 (cvss_score): 0.0135999436186787

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.74      0.71      0.73        28
         1.0       0.98      0.86      0.91        50
         2.0       0.81      0.82      0.81       103
         3.0       0.85      0.83      0.84       200
         4.0       0.92      0.91      0.91       834
         5.0       0.83      0.93      0.87        41
         6.0       0.49      0.53      0.51        36
         7.0       0.85      0.86      0.85       155
         8.0       0.98      1.00      0.99       176
         9.0       0.87      0.93      0.90        14
        10.0       0.95      0.95      0.95       423

    accuracy  

# Random Forest

In [11]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
multi_rf = MultiOutputClassifier(rf)

multi_rf.fit(x_train, y_train)

importances = np.mean([est.feature_importances_ for est in multi_rf.estimators_], axis=0)
selected_features = x_train.columns[importances > np.mean(importances)]

multi_rf.fit(x_train[selected_features], y_train)
y_pred = multi_rf.predict(x_test[selected_features])

rf_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
rf_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

rf_scores_type = cross_val_score(rf, x[selected_features], y['type'], cv=kf, scoring='f1_macro')
rf_scores_cvss_score  = cross_val_score(rf, x[selected_features], y['cvss_score'], cv=kf, scoring='f1_macro')

print("Random Forest Accuracy for 'type':", rf_accuracy_type)
print("Random Forest Accuracy for 'cvss_score' :", rf_accuracy_cvss_score)

print("K-Fold mean F1 (type):", rf_scores_type.mean())
print("K-Fold std  F1 (type):", rf_scores_type.std())

print("K-Fold mean F1 (cvss_score):", rf_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", rf_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Random Forest Accuracy for 'type': 0.8776699029126214
Random Forest Accuracy for 'cvss_score' : 0.6815533980582524
K-Fold mean F1 (type): 0.6786973747481492
K-Fold std  F1 (type): 0.007657113318103228
K-Fold mean F1 (cvss_score): 0.5442442506227819
K-Fold std  F1 (cvss_score): 0.007055799496073992

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       1.00      0.07      0.13        28
         1.0       0.98      0.82      0.89        50
         2.0       0.78      0.46      0.58       103
         3.0       0.77      0.89      0.82       200
         4.0       0.84      0.96      0.90       834
         5.0       0.93      0.63      0.75        41
         6.0       0.75      0.08      0.15        36
         7.0       0.89      0.83      0.86       155
         8.0       1.00      0.99      0.99       176
         9.0       1.00      0.36      0.53        14
        10.0       0.97      0.95      0.96       423

    accuracy   

# Gradient Boosting

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
multi_gbr = MultiOutputClassifier(gb)

multi_gbr.fit(x_train, y_train)

importances = np.mean([est.feature_importances_ for est in multi_gbr.estimators_], axis=0)
selected_features = x_train.columns[importances > np.mean(importances)]

multi_gbr.fit(x_train[selected_features], y_train)
y_pred = multi_gbr.predict(x_test[selected_features])

gb_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
gb_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

gb_scores_type = cross_val_score(gb, x[selected_features], y['type'], cv=kf, scoring='f1_macro')
gb_scores_cvss_score  = cross_val_score(gb, x[selected_features], y['cvss_score'], cv=kf, scoring='f1_macro')

print("Gradient Boosting Accuracy for 'type':", gb_accuracy_type)
print("Gradient Boosting Accuracy for 'cvss_score' :", gb_accuracy_cvss_score)

print("K-Fold mean F1 (type):", gb_scores_type.mean())
print("K-Fold std  F1 (type):", gb_scores_type.std())

print("K-Fold mean F1 (cvss_score):", gb_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", gb_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Gradient Boosting Accuracy for 'type': 0.9296116504854369
Gradient Boosting Accuracy for 'cvss_score' : 0.6480582524271845
K-Fold mean F1 (type): 0.8832641900418396
K-Fold std  F1 (type): 0.006370413942588808
K-Fold mean F1 (cvss_score): 0.518344292076997
K-Fold std  F1 (cvss_score): 0.006949892293444006

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.88      0.79      0.83        28
         1.0       0.98      0.84      0.90        50
         2.0       0.82      0.96      0.88       103
         3.0       0.93      0.89      0.91       200
         4.0       0.92      0.94      0.93       834
         5.0       0.95      0.93      0.94        41
         6.0       0.83      0.56      0.67        36
         7.0       0.89      0.85      0.86       155
         8.0       0.99      0.98      0.99       176
         9.0       0.88      1.00      0.93        14
        10.0       0.97      0.99      0.98       423

    accu

# Extra Tree

In [13]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(random_state=42)
multi_et = MultiOutputClassifier(et)

multi_et.fit(x_train, y_train)

importances = np.mean([est.feature_importances_ for est in multi_et.estimators_], axis=0)
selected_features = x_train.columns[importances > np.mean(importances)]

multi_et.fit(x_train[selected_features], y_train)
y_pred = multi_et.predict(x_test[selected_features])

et_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
et_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

et_scores_type = cross_val_score(et, x[selected_features], y['type'], cv=kf, scoring='f1_macro')
et_scores_cvss_score  = cross_val_score(et, x[selected_features], y['cvss_score'], cv=kf, scoring='f1_macro')

print("Extra Trees Accuracy for 'type':", et_accuracy_type)
print("Extra Trees Accuracy for 'cvss_score' :", et_accuracy_cvss_score)

print("K-Fold mean F1 (type):", et_scores_type.mean())
print("K-Fold std  F1 (type):", et_scores_type.std())

print("K-Fold mean F1 (cvss_score):", et_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", et_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Extra Trees Accuracy for 'type': 0.8844660194174757
Extra Trees Accuracy for 'cvss_score' : 0.6747572815533981
K-Fold mean F1 (type): 0.7035314423420451
K-Fold std  F1 (type): 0.014804209292476082
K-Fold mean F1 (cvss_score): 0.5506485639942478
K-Fold std  F1 (cvss_score): 0.017242510035925297

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.75      0.11      0.19        28
         1.0       0.98      0.80      0.88        50
         2.0       0.82      0.95      0.88       103
         3.0       0.75      0.79      0.77       200
         4.0       0.86      0.95      0.91       834
         5.0       0.92      0.59      0.72        41
         6.0       0.89      0.22      0.36        36
         7.0       0.87      0.81      0.84       155
         8.0       0.99      0.99      0.99       176
         9.0       1.00      0.07      0.13        14
        10.0       0.97      0.94      0.96       423

    accuracy       

# Hist Gradient Boosting

In [14]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
multi_hgb = MultiOutputClassifier(hgb)

multi_hgb.fit(x_train, y_train)
y_pred = multi_hgb.predict(x_test)

hgb_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
hgb_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

hgb_scores_type = cross_val_score(hgb, x[selected_features], y['type'], cv=kf, scoring='f1_macro')
hgb_scores_cvss_score  = cross_val_score(hgb, x[selected_features], y['cvss_score'], cv=kf, scoring='f1_macro')

print("HistGradientBoosting Accuracy for 'type':", hgb_accuracy_type)
print("HistGradientBoosting Accuracy for 'cvss_score' :", hgb_accuracy_cvss_score)

print("K-Fold mean F1 (type):", hgb_scores_type.mean())
print("K-Fold std  F1 (type):", hgb_scores_type.std())

print("K-Fold mean F1 (cvss_score):", hgb_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", hgb_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

HistGradientBoosting Accuracy for 'type': 0.9495145631067962
HistGradientBoosting Accuracy for 'cvss_score' : 0.6805825242718446
K-Fold mean F1 (type): 0.859650400572609
K-Fold std  F1 (type): 0.0005043169989493918
K-Fold mean F1 (cvss_score): 0.5529562162823665
K-Fold std  F1 (cvss_score): 0.012298547389334093

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       1.00      0.89      0.94        28
         1.0       0.98      0.96      0.97        50
         2.0       0.85      0.91      0.88       103
         3.0       0.94      0.93      0.94       200
         4.0       0.95      0.96      0.95       834
         5.0       0.95      0.93      0.94        41
         6.0       0.82      0.89      0.85        36
         7.0       0.90      0.84      0.87       155
         8.0       0.99      0.99      0.99       176
         9.0       1.00      1.00      1.00        14
        10.0       0.99      0.98      0.98       423

 

# KNN

In [15]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=10)
multi_knn = MultiOutputClassifier(knn)

multi_knn.fit(x_train, y_train)
y_pred = multi_knn.predict(x_test)

knn_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
knn_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

knn_scores_type = cross_val_score(knn, x[selected_features], y['type'], cv=kf, scoring='f1_macro')
knn_scores_cvss_score  = cross_val_score(knn, x[selected_features], y['cvss_score'], cv=kf, scoring='f1_macro')

print("KNN Accuracy for 'type':", knn_accuracy_type)
print("KNN Accuracy for 'cvss_score' :", knn_accuracy_cvss_score)

print("K-Fold mean F1 (type):", knn_scores_type.mean())
print("K-Fold std  F1 (type):", knn_scores_type.std())

print("K-Fold mean F1 (cvss_score):", knn_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", knn_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

KNN Accuracy for 'type': 0.8004854368932038
KNN Accuracy for 'cvss_score' : 0.6121359223300971
K-Fold mean F1 (type): 0.5419164462924732
K-Fold std  F1 (type): 0.002867879009922753
K-Fold mean F1 (cvss_score): 0.4973094519432144
K-Fold std  F1 (cvss_score): 0.011297243174919617

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.50      0.04      0.07        28
         1.0       0.83      0.78      0.80        50
         2.0       0.85      0.67      0.75       103
         3.0       0.55      0.66      0.60       200
         4.0       0.76      0.89      0.82       834
         5.0       0.96      0.56      0.71        41
         6.0       0.71      0.14      0.23        36
         7.0       0.88      0.61      0.72       155
         8.0       1.00      0.94      0.97       176
         9.0       1.00      0.21      0.35        14
        10.0       0.93      0.89      0.91       423

    accuracy                       

# Adaboost

In [16]:
from sklearn.ensemble import AdaBoostClassifier

ab = AdaBoostClassifier(random_state=42)
multi_ada = MultiOutputClassifier(ab)

multi_ada.fit(x_train, y_train)

importances = np.mean([est.feature_importances_ for est in multi_ada.estimators_], axis=0)
selected_features = x_train.columns[importances > np.mean(importances)]

multi_ada.fit(x_train[selected_features], y_train)
y_pred = multi_ada.predict(x_test[selected_features])

ab_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
ab_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

ab_scores_type = cross_val_score(ab, x[selected_features], y['type'], cv=kf, scoring='f1_macro')
ab_scores_cvss_score  = cross_val_score(ab, x[selected_features], y['cvss_score'], cv=kf, scoring='f1_macro')

print("AdaBoost Accuracy for 'type':", ab_accuracy_type)
print("AdaBoost Accuracy for 'cvss_score' :", ab_accuracy_cvss_score)

print("K-Fold mean F1 (type):", ab_scores_type.mean())
print("K-Fold std  F1 (type):", ab_scores_type.std())

print("K-Fold mean F1 (cvss_score):", ab_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", ab_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

AdaBoost Accuracy for 'type': 0.7126213592233009
AdaBoost Accuracy for 'cvss_score' : 0.5849514563106796
K-Fold mean F1 (type): 0.43684599452652995
K-Fold std  F1 (type): 0.054786432642828814
K-Fold mean F1 (cvss_score): 0.2806172113398212
K-Fold std  F1 (cvss_score): 0.0021172211937914463

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        28
         1.0       0.71      0.80      0.75        50
         2.0       0.63      0.74      0.68       103
         3.0       0.09      0.03      0.04       200
         4.0       0.64      0.91      0.76       834
         5.0       0.00      0.00      0.00        41
         6.0       0.00      0.00      0.00        36
         7.0       0.72      0.45      0.55       155
         8.0       1.00      0.74      0.85       176
         9.0       0.00      0.00      0.00        14
        10.0       0.95      0.91      0.93       423

    accuracy           

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classif

# LightGBM

In [17]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(random_state=42)
multi_lgbm = MultiOutputClassifier(lgbm)

multi_lgbm.fit(x_train, y_train)

importances = np.mean([est.feature_importances_ for est in multi_lgbm.estimators_], axis=0)
selected_features = x_train.columns[importances > np.mean(importances)]

multi_lgbm.fit(x_train[selected_features], y_train)
y_pred = multi_lgbm.predict(x_test[selected_features])

lgbm_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
lgbm_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

lgbm_scores_type = cross_val_score(lgbm, x[selected_features], y['type'], cv=kf, scoring='f1_macro')
lgbm_scores_cvss_score  = cross_val_score(lgbm, x[selected_features], y['cvss_score'], cv=kf, scoring='f1_macro')

print("LightGBM Accuracy for 'type':", lgbm_accuracy_type)
print("LightGBM Accuracy for 'cvss_score' :", lgbm_accuracy_cvss_score)

print("K-Fold mean F1 (type):", lgbm_scores_type.mean())
print("K-Fold std  F1 (type):", lgbm_scores_type.std())

print("K-Fold mean F1 (cvss_score):", lgbm_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", lgbm_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001279 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2988
[LightGBM] [Info] Number of data points in the train set: 8236, number of used features: 27
[LightGBM] [Info] Start training from score -4.597429
[LightGBM] [Info] Start training from score -3.712965
[LightGBM] [Info] Start training from score -2.891587
[LightGBM] [Info] Start training from score -2.335415
[LightGBM] [Info] Start training from score -0.856323
[LightGBM] [Info] Start training from score -3.645632
[LightGBM] [Info] Start training from score -4.019058
[LightGBM] [Info] Start training from score -2.611042
[LightGBM] [Info] Start training from score -2.547020
[LightGBM] [Info] Start training from score -4.767775
[LightGBM] [Info] Start training from score -1.691780
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

# Bagging

In [18]:
from sklearn.ensemble import BaggingClassifier

bagging = BaggingClassifier(random_state=42)
multi_bagging = MultiOutputClassifier(bagging)

multi_bagging.fit(x_train, y_train)
y_pred = multi_bagging.predict(x_test)

bag_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
bag_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

bag_scores_type = cross_val_score(bagging, x[selected_features], y['type'], cv=kf, scoring='f1_macro')
bag_scores_cvss_score  = cross_val_score(bagging, x[selected_features], y['cvss_score'], cv=kf, scoring='f1_macro')

print(f'Bagging Classifier Accuracy for "type": {bag_accuracy_type}')
print(f'Bagging Classifier Accuracy for "cvss_score" : {bag_accuracy_cvss_score}')

print("K-Fold mean F1 (type):", bag_scores_type.mean())
print("K-Fold std  F1 (type):", bag_scores_type.std())

print("K-Fold mean F1 (cvss_score):", bag_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", bag_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Bagging Classifier Accuracy for "type": 0.9446601941747573
Bagging Classifier Accuracy for "cvss_score" : 0.6495145631067961
K-Fold mean F1 (type): 0.7984691475527715
K-Fold std  F1 (type): 0.014819302691681416
K-Fold mean F1 (cvss_score): 0.5174672441015332
K-Fold std  F1 (cvss_score): 0.013548073649350388

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       1.00      0.86      0.92        28
         1.0       1.00      0.94      0.97        50
         2.0       0.82      0.94      0.87       103
         3.0       0.97      0.91      0.94       200
         4.0       0.94      0.96      0.95       834
         5.0       0.97      0.88      0.92        41
         6.0       0.78      0.89      0.83        36
         7.0       0.90      0.86      0.88       155
         8.0       0.98      1.00      0.99       176
         9.0       0.82      1.00      0.90        14
        10.0       0.98      0.97      0.97       423

    a

# SVC

In [19]:
from sklearn.svm import SVC

svc = SVC(kernel='rbf', C=5, probability=True)
multi_svc = MultiOutputClassifier(svc)

multi_svc.fit(x_train[selected_features], y_train)
y_pred = multi_svc.predict(x_test[selected_features])

svc_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
svc_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

svc_scores_type = cross_val_score(svc, x[selected_features], y['type'], cv=kf, scoring='f1_macro')
svc_scores_cvss_score  = cross_val_score(svc, x[selected_features], y['cvss_score'], cv=kf, scoring='f1_macro')

print(f'SVC Accuracy for "type": {svc_accuracy_type}')
print(f'SVC Accuracy for "cvss_score" : {svc_accuracy_cvss_score}')

print("K-Fold mean F1 (type):", svc_scores_type.mean())
print("K-Fold std  F1 (type):", svc_scores_type.std())

print("K-Fold mean F1 (cvss_score):", svc_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", svc_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

SVC Accuracy for "type": 0.6087378640776699
SVC Accuracy for "cvss_score" : 0.579126213592233
K-Fold mean F1 (type): 0.320467579315816
K-Fold std  F1 (type): 0.0066060380923177965
K-Fold mean F1 (cvss_score): 0.26584172063699024
K-Fold std  F1 (cvss_score): 0.0067490062036051806

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        28
         1.0       0.53      0.66      0.59        50
         2.0       0.67      0.06      0.11       103
         3.0       0.50      0.69      0.58       200
         4.0       0.59      0.85      0.70       834
         5.0       0.00      0.00      0.00        41
         6.0       0.00      0.00      0.00        36
         7.0       0.60      0.37      0.46       155
         8.0       0.60      0.40      0.48       176
         9.0       0.00      0.00      0.00        14
        10.0       0.80      0.58      0.67       423

    accuracy                      

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classif

# Hard Voting

In [20]:
from sklearn.ensemble import VotingClassifier

rf_fs = RandomForestClassifier(n_estimators=300, random_state=42)
multi_rf_fs = MultiOutputClassifier(rf_fs)

multi_rf_fs.fit(x_train, y_train)

importances = np.mean([est.feature_importances_ for est in multi_rf_fs.estimators_], axis=0)
selected_features = x_train.columns[importances > np.mean(importances)]

model1 = RandomForestClassifier(n_estimators=200, random_state=42)
model2 = ExtraTreesClassifier(n_estimators=200, random_state=42)
model3 = LogisticRegression(max_iter=500)

voting_hard = VotingClassifier(
    estimators=[
        ('rf', model1),
        ('et', model2),
        ('lr', model3)
    ],
    voting='hard'
)

multi_voting_hard = MultiOutputClassifier(voting_hard)

multi_voting_hard.fit(x_train[selected_features], y_train)
y_pred = multi_voting_hard.predict(x_test[selected_features])

hard_acc_type = accuracy_score(y_test["type"], y_pred[:, 0])
hard_acc_cvss_score  = accuracy_score(y_test["cvss_score"],  y_pred[:, 1])

hard_scores_type = cross_val_score(voting_hard, x[selected_features], y['type'], cv=kf, scoring='f1_macro')
hard_scores_cvss_score  = cross_val_score(voting_hard, x[selected_features], y['cvss_score'],  cv=kf, scoring='f1_macro')

print("Hard Voting Accuracy for type:", hard_acc_type)
print("Hard Voting Accuracy for cvss_score :", hard_acc_cvss_score)

print("K-fold F1 mean (type):", hard_scores_type.mean())
print("K-fold F1 std  (type):", hard_scores_type.std())

print("K-fold F1 mean (cvss_score) :", hard_scores_cvss_score.mean())
print("K-fold F1 std  (cvss_score) :", hard_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test["type"], y_pred[:, 0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test["cvss_score"],  y_pred[:, 1]))

Hard Voting Accuracy for type: 0.8684466019417476
Hard Voting Accuracy for cvss_score : 0.6728155339805825
K-fold F1 mean (type): 0.6620942774664145
K-fold F1 std  (type): 0.010224171572265451
K-fold F1 mean (cvss_score) : 0.5383844188044781
K-fold F1 std  (cvss_score) : 0.0062594570041507955

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       1.00      0.07      0.13        28
         1.0       0.95      0.78      0.86        50
         2.0       0.77      0.45      0.56       103
         3.0       0.76      0.83      0.79       200
         4.0       0.82      0.97      0.89       834
         5.0       1.00      0.56      0.72        41
         6.0       0.75      0.08      0.15        36
         7.0       0.91      0.81      0.85       155
         8.0       1.00      0.99      0.99       176
         9.0       1.00      0.21      0.35        14
        10.0       0.97      0.95      0.96       423

    accuracy        

# Soft Voting

In [21]:
from sklearn.ensemble import VotingClassifier

rf_fs = RandomForestClassifier(n_estimators=300, random_state=42)
multi_rf_fs = MultiOutputClassifier(rf_fs)

multi_rf_fs.fit(x_train, y_train)

importances = np.mean([est.feature_importances_ for est in multi_rf_fs.estimators_], axis=0)
selected_features = x_train.columns[importances > np.mean(importances)]

model1 = RandomForestClassifier(n_estimators=200, random_state=42)
model2 = ExtraTreesClassifier(n_estimators=200, random_state=42)
model3 = LogisticRegression(max_iter=500)

voting_soft = VotingClassifier(
    estimators=[
        ('rf', model1),
        ('et', model2),
        ('lr', model3)
    ],
    voting='soft'
)

multi_voting_soft = MultiOutputClassifier(voting_soft)

multi_voting_soft.fit(x_train[selected_features], y_train)
y_pred = multi_voting_soft.predict(x_test[selected_features])

soft_acc_type = accuracy_score(y_test["type"], y_pred[:, 0])
soft_acc_cvss_score  = accuracy_score(y_test["cvss_score"],  y_pred[:, 1])

soft_scores_type = cross_val_score(voting_soft, x[selected_features], y['type'], cv=kf, scoring='f1_macro')
soft_scores_cvss_score  = cross_val_score(voting_soft, x[selected_features], y['cvss_score'],  cv=kf, scoring='f1_macro')

print("Soft Voting Accuracy for type:", soft_acc_type)
print("Soft Voting Accuracy for cvss_score :", soft_acc_cvss_score)

print("K-fold F1 mean (type):", soft_scores_type.mean())
print("K-fold F1 std  (type):", soft_scores_type.std())

print("K-fold F1 mean (cvss_score) :", soft_scores_cvss_score.mean())
print("K-fold F1 std  (cvss_score) :", soft_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test["type"], y_pred[:, 0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test["cvss_score"],  y_pred[:, 1]))

Soft Voting Accuracy for type: 0.858252427184466
Soft Voting Accuracy for cvss_score : 0.670873786407767
K-fold F1 mean (type): 0.6141315241406103
K-fold F1 std  (type): 0.007163728409382124
K-fold F1 mean (cvss_score) : 0.5155132108741932
K-fold F1 std  (cvss_score) : 0.00968546053559784

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       1.00      0.04      0.07        28
         1.0       0.98      0.80      0.88        50
         2.0       0.80      0.36      0.50       103
         3.0       0.74      0.81      0.78       200
         4.0       0.81      0.97      0.88       834
         5.0       0.96      0.54      0.69        41
         6.0       1.00      0.06      0.11        36
         7.0       0.87      0.79      0.83       155
         8.0       1.00      0.99      0.99       176
         9.0       0.00      0.00      0.00        14
        10.0       0.97      0.95      0.96       423

    accuracy            

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classif

# Stacking

In [22]:
from sklearn.ensemble import StackingClassifier

rf_fs = RandomForestClassifier(n_estimators=300, random_state=42)
multi_rf_fs = MultiOutputClassifier(rf_fs)
multi_rf_fs.fit(x_train, y_train)

importances = np.mean([est.feature_importances_ for est in multi_rf_fs.estimators_], axis=0)
selected_features = x_train.columns[importances > np.mean(importances)]

base1 = RandomForestClassifier(n_estimators=200, random_state=42)
base2 = ExtraTreesClassifier(n_estimators=200, random_state=42)
base3 = LogisticRegression(max_iter=500)

stacking = StackingClassifier(
    estimators=[
        ('rf', base1),
        ('et', base2),
        ('lr', base3)
    ],
    final_estimator=LogisticRegression(max_iter=500)
)

multi_stacking = MultiOutputClassifier(stacking)
multi_stacking.fit(x_train[selected_features], y_train)
y_pred = multi_stacking.predict(x_test[selected_features])

stacking_acc_type = accuracy_score(y_test["type"], y_pred[:, 0])
stacking_acc_cvss_score  = accuracy_score(y_test["cvss_score"],  y_pred[:, 1])

stacking_scores_type = cross_val_score(stacking, x[selected_features], y['type'], cv=kf, scoring='f1_macro')
stacking_scores_cvss_score  = cross_val_score(stacking, x[selected_features], y['cvss_score'],  cv=kf, scoring='f1_macro')

print("Stacking Accuracy for type:", stacking_acc_type)
print("Stacking Accuracy for cvss_score :", stacking_acc_cvss_score)
print("K-fold F1 mean (type):", stacking_scores_type.mean())
print("K-fold F1 std  (type):", stacking_scores_type.std())
print("K-fold F1 mean (cvss_score) :", stacking_scores_cvss_score.mean())
print("K-fold F1 std  (cvss_score) :", stacking_scores_cvss_score.std())
print("\nClassification Report for 'type':\n", classification_report(y_test["type"], y_pred[:, 0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test["cvss_score"], y_pred[:, 1]))

Stacking Accuracy for type: 0.8849514563106796
Stacking Accuracy for cvss_score : 0.6776699029126214
K-fold F1 mean (type): 0.73703770188323
K-fold F1 std  (type): 0.012695507400019512
K-fold F1 mean (cvss_score) : 0.5484020365417034
K-fold F1 std  (cvss_score) : 0.012797966442802915

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.57      0.14      0.23        28
         1.0       0.98      0.84      0.90        50
         2.0       0.72      0.48      0.57       103
         3.0       0.80      0.87      0.83       200
         4.0       0.86      0.96      0.91       834
         5.0       0.89      0.76      0.82        41
         6.0       0.70      0.19      0.30        36
         7.0       0.87      0.83      0.85       155
         8.0       1.00      0.99      0.99       176
         9.0       1.00      0.57      0.73        14
        10.0       0.97      0.96      0.97       423

    accuracy                 

# Bagged KNN

In [ ]:
bag_knn = BaggingClassifier(
    estimator=KNeighborsClassifier(),
    n_estimators=100,
    random_state=42
)

multi_bag_knn = MultiOutputClassifier(bag_knn)

multi_bag_knn.fit(x_train, y_train)
y_pred = multi_bag_knn.predict(x_test)

bag_knn_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
bag_knn_accuracy_cvss_score = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

bag_knn_scores_type = cross_val_score(bag_knn, x_train, y_train['type'], cv=kf, scoring='f1_macro')
bag_knn_scores_cvss_score = cross_val_score(bag_knn, x_train, y_train['cvss_score'], cv=kf, scoring='f1_macro')

print(f'Bagging KNN Accuracy for "type": {bag_knn_accuracy_type}')
print(f'Bagging KNN Accuracy for "cvss_score": {bag_knn_accuracy_cvss_score}')

print("K-Fold mean F1 (type):", bag_knn_scores_type.mean())
print("K-Fold std  F1 (type):", bag_knn_scores_type.std())

print("K-Fold mean F1 (cvss_score):", bag_knn_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", bag_knn_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Bagging KNN Accuracy for "type": 0.8121359223300971
Bagging KNN Accuracy for "cvss_score": 0.6242718446601941
K-Fold mean F1 (type): 0.4772369616127343
K-Fold std  F1 (type): 0.011757416078566299
K-Fold mean F1 (cvss_score): 0.46596920817337417
K-Fold std  F1 (cvss_score): 0.00621545773156151

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.33      0.07      0.12        28
         1.0       0.85      0.78      0.81        50
         2.0       0.83      0.71      0.76       103
         3.0       0.56      0.66      0.61       200
         4.0       0.78      0.89      0.83       834
         5.0       0.96      0.59      0.73        41
         6.0       0.78      0.19      0.31        36
         7.0       0.83      0.65      0.73       155
         8.0       0.98      0.95      0.97       176
         9.0       1.00      0.43      0.60        14
        10.0       0.93      0.91      0.92       423

    accuracy        

# Bagged DecisionTree

In [25]:
bag_dt = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    random_state=42
)

multi_bag_dt = MultiOutputClassifier(bag_dt)

multi_bag_dt.fit(x_train, y_train)

multi_bag_dt.fit(x_train, y_train)
y_pred = multi_bag_dt.predict(x_test)

bag_dt_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
bag_dt_accuracy_cvss_score = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

bag_dt_scores_type = cross_val_score(bag_dt, x_train, y_train['type'], cv=kf, scoring='f1_macro')
bag_dt_scores_cvss_score = cross_val_score(bag_dt, x_train, y_train['cvss_score'], cv=kf, scoring='f1_macro')

print("Bagging DecisionTree Accuracy for 'type':", bag_dt_accuracy_type)
print("Bagging DecisionTree Accuracy for 'cvss_score':", bag_dt_accuracy_cvss_score)

print("K-Fold mean F1 (type):", bag_dt_scores_type.mean())
print("K-Fold std  F1 (type):", bag_dt_scores_type.std())

print("K-Fold mean F1 (cvss_score):", bag_dt_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", bag_dt_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Bagging DecisionTree Accuracy for 'type': 0.95
Bagging DecisionTree Accuracy for 'cvss_score': 0.6694174757281554
K-Fold mean F1 (type): 0.9243748215703596
K-Fold std  F1 (type): 0.006297110023684206
K-Fold mean F1 (cvss_score): 0.5389630682376138
K-Fold std  F1 (cvss_score): 0.020637064502082584

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       1.00      0.86      0.92        28
         1.0       1.00      0.96      0.98        50
         2.0       0.82      0.91      0.87       103
         3.0       0.98      0.93      0.95       200
         4.0       0.95      0.96      0.95       834
         5.0       0.97      0.90      0.94        41
         6.0       0.79      0.92      0.85        36
         7.0       0.90      0.88      0.89       155
         8.0       0.99      1.00      1.00       176
         9.0       0.82      1.00      0.90        14
        10.0       0.98      0.97      0.98       423

    accuracy    

In [26]:
from rich.table import Table
from rich.console import Console

console = Console()

results = [
    ['LogisticRegression', lr_accuracy_type, lr_scores_type.mean(), lr_scores_type.std(), lr_accuracy_cvss_score, lr_scores_cvss_score.mean(), lr_scores_cvss_score.std()],
    ['DecisionTree', dt_accuracy_type, dt_scores_type.mean(), dt_scores_type.std(), dt_accuracy_cvss_score, dt_scores_cvss_score.mean(), dt_scores_cvss_score.std()],
    ['RandomForest', rf_accuracy_type, rf_scores_type.mean(), rf_scores_type.std(), rf_accuracy_cvss_score, rf_scores_cvss_score.mean(), rf_scores_cvss_score.std()],
    ['ExtraTrees', et_accuracy_type, et_scores_type.mean(), et_scores_type.std(), et_accuracy_cvss_score, et_scores_cvss_score.mean(), et_scores_cvss_score.std()],
    ['GradientBoosting', gb_accuracy_type, gb_scores_type.mean(), gb_scores_type.std(), gb_accuracy_cvss_score, gb_scores_cvss_score.mean(), gb_scores_cvss_score.std()],
    ['HistGradientBoosting', hgb_accuracy_type, hgb_scores_type.mean(), hgb_scores_type.std(), hgb_accuracy_cvss_score, hgb_scores_cvss_score.mean(), hgb_scores_cvss_score.std()],
    ['KNN', knn_accuracy_type, knn_scores_type.mean(), knn_scores_type.std(), knn_accuracy_cvss_score, knn_scores_cvss_score.mean(), knn_scores_cvss_score.std()],
    ['AdaBoost', ab_accuracy_type, ab_scores_type.mean(), ab_scores_type.std(), ab_accuracy_cvss_score, ab_scores_cvss_score.mean(), ab_scores_cvss_score.std()],
    ['LightGBM', lgbm_accuracy_type, lgbm_scores_type.mean(), lgbm_scores_type.std(), lgbm_accuracy_cvss_score, lgbm_scores_cvss_score.mean(), lgbm_scores_cvss_score.std()],
    ['Bagging', bag_accuracy_type, bag_scores_type.mean(), bag_scores_type.std(), bag_accuracy_cvss_score, bag_scores_cvss_score.mean(), bag_scores_cvss_score.std()],
    ['Hard Voting', hard_acc_type, hard_scores_type.mean(), hard_scores_type.std(), hard_acc_cvss_score, hard_scores_cvss_score.mean(), hard_scores_cvss_score.std()],
    ['Soft Voting', soft_acc_type, soft_scores_type.mean(), soft_scores_type.std(), soft_acc_cvss_score, soft_scores_cvss_score.mean(), soft_scores_cvss_score.std()],
    ['Stacking', stacking_acc_type, stacking_scores_type.mean(), stacking_scores_type.std(), stacking_acc_cvss_score, stacking_scores_cvss_score.mean(), stacking_scores_cvss_score.std()],
    ['SVM', svc_accuracy_type, svc_scores_type.mean(), svc_scores_type.std(), svc_accuracy_cvss_score, svc_scores_cvss_score.mean(), svc_scores_cvss_score.std()],
    ['Bagged KNN', bag_knn_accuracy_type, bag_knn_scores_type.mean(), bag_knn_scores_type.std(), bag_knn_accuracy_cvss_score, bag_knn_scores_cvss_score.mean(), bag_knn_scores_cvss_score.std()],
    ['Bagged DT', bag_dt_accuracy_type, bag_dt_scores_type.mean(), bag_dt_scores_type.std(), bag_dt_accuracy_cvss_score, bag_dt_scores_cvss_score.mean(), bag_dt_scores_cvss_score.std()],
]


for row in results:
    type_acc = row[1]
    cvss_score_acc = row[4]
    combined = (type_acc + cvss_score_acc) / 2
    row.append(combined)

result_sorted = sorted(results, key=lambda i: i[-1], reverse=True)

best_model = max(results, key=lambda x: x[-1])
worst_model = min(results, key=lambda x: x[-1])

table = Table(title="Embedded Method Comparison", show_lines=True)
table.add_column("Algorithm")
table.add_column("Type Acc")
table.add_column("K-Fold Mean")
table.add_column("K-Fold Std")
table.add_column("cvss_score Acc")
table.add_column("K-Fold Mean")
table.add_column("K-Fold Std")
table.add_column("Combined", justify="right")

for row in result_sorted:
    algo, type_acc, kmean_type, kstd_type, cvss_score_acc, kmean_cvss_score, kstd_cvss_score, combined = row

    if row == best_model:
        table.add_row(
            f"[bold green]{algo}[/bold green]",
            f"[bold green]{type_acc:.2f}[/bold green]",
            f"[bold green]{kmean_type:.2f}[/bold green]",
            f"[bold green]{kstd_type:.2f}[/bold green]",
            f"[bold green]{cvss_score_acc:.2f}[/bold green]",
            f"[bold green]{kmean_cvss_score:.2f}[/bold green]",
            f"[bold green]{kstd_cvss_score:.2f}[/bold green]",
            f"[bold green]{combined:.2f}[/bold green]",
        )
    elif row == worst_model:
        table.add_row(
            f"[bold red]{algo}[/bold red]",
            f"[bold red]{type_acc:.2f}[/bold red]",
            f"[bold red]{kmean_type:.2f}[/bold red]",
            f"[bold red]{kstd_type:.2f}[/bold red]",
            f"[bold red]{cvss_score_acc:.2f}[/bold red]",
            f"[bold red]{kmean_cvss_score:.2f}[/bold red]",
            f"[bold red]{kstd_cvss_score:.2f}[/bold red]",
            f"[bold red]{combined:.2f}[/bold red]",
        )
    else:
        table.add_row(
            algo, f"{type_acc:.2f}", f"{kmean_type:.2f}", f"{kstd_type:.2f}",
            f"{cvss_score_acc:.2f}", f"{kmean_cvss_score:.2f}", f"{kstd_cvss_score:.2f}", f"{combined:.2f}"
        )

console.print(table)


                                            Embedded Method Comparison                                             
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Algorithm          ┃ Type Acc ┃ K-Fold Mean ┃ K-Fold Std ┃ cvss_score Acc ┃ K-Fold Mean ┃ K-Fold Std ┃ Combined ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ HistGradientBoost… │ 0.95     │ 0.86        │ 0.00       │ 0.68           │ 0.55        │ 0.01       │     0.82 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagged DT          │ 0.95     │ 0.92        │ 0.01       │ 0.67           │ 0.54        │ 0.02       │     0.81 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagging            │ 0.94     │ 0.80        │ 0.01       │ 0.65           │ 0.52        │ 0.01       │     0.80 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ GradientBoosting   │ 0.93     │ 0.88        │ 0.01       │ 0.65           │ 0.52        │ 0.01       │     0.79 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Stacking           │ 0.88     │ 0.74        │ 0.01       │ 0.68           │ 0.55        │ 0.01       │     0.78 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ RandomForest       │ 0.88     │ 0.68        │ 0.01       │ 0.68           │ 0.54        │ 0.01       │     0.78 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ ExtraTrees         │ 0.88     │ 0.70        │ 0.01       │ 0.67           │ 0.55        │ 0.02       │     0.78 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ LightGBM           │ 0.90     │ 0.83        │ 0.00       │ 0.65           │ 0.53        │ 0.01       │     0.78 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Hard Voting        │ 0.87     │ 0.66        │ 0.01       │ 0.67           │ 0.54        │ 0.01       │     0.77 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Soft Voting        │ 0.86     │ 0.61        │ 0.01       │ 0.67           │ 0.52        │ 0.01       │     0.76 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ DecisionTree       │ 0.90     │ 0.84        │ 0.01       │ 0.59           │ 0.46        │ 0.01       │     0.75 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagged KNN         │ 0.81     │ 0.48        │ 0.01       │ 0.62           │ 0.47        │ 0.01       │     0.72 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ KNN                │ 0.80     │ 0.54        │ 0.00       │ 0.61           │ 0.50        │ 0.01       │     0.71 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ LogisticRegression │ 0.75     │ 0.59        │ 0.01       │ 0.59           │ 0.28        │ 0.00       │     0.67 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ AdaBoost           │ 0.71     │ 0.44        │ 0.05       │ 0.58           │ 0.28        │ 0.00       │     0.65 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ SVM                │ 0.61     │ 0.32        │ 0.01    

In [27]:
os.makedirs('results', exist_ok=True)

temp_console = Console(record=True)
temp_console.print(table)
text = temp_console.export_text()
with open('results/feature_selection_compare.txt', 'a', encoding='utf-8') as f:
    f.write(text)

                                            Embedded Method Comparison                                             
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Algorithm          ┃ Type Acc ┃ K-Fold Mean ┃ K-Fold Std ┃ cvss_score Acc ┃ K-Fold Mean ┃ K-Fold Std ┃ Combined ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ HistGradientBoost… │ 0.95     │ 0.86        │ 0.00       │ 0.68           │ 0.55        │ 0.01       │     0.82 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagged DT          │ 0.95     │ 0.92        │ 0.01       │ 0.67           │ 0.54        │ 0.02       │     0.81 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagging            │ 0.94     │ 0.80        │ 0.01       │ 0.65           │ 0.52        │ 0.01       │     0.80 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ GradientBoosting   │ 0.93     │ 0.88        │ 0.01       │ 0.65           │ 0.52        │ 0.01       │     0.79 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Stacking           │ 0.88     │ 0.74        │ 0.01       │ 0.68           │ 0.55        │ 0.01       │     0.78 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ RandomForest       │ 0.88     │ 0.68        │ 0.01       │ 0.68           │ 0.54        │ 0.01       │     0.78 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ ExtraTrees         │ 0.88     │ 0.70        │ 0.01       │ 0.67           │ 0.55        │ 0.02       │     0.78 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ LightGBM           │ 0.90     │ 0.83        │ 0.00       │ 0.65           │ 0.53        │ 0.01       │     0.78 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Hard Voting        │ 0.87     │ 0.66        │ 0.01       │ 0.67           │ 0.54        │ 0.01       │     0.77 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Soft Voting        │ 0.86     │ 0.61        │ 0.01       │ 0.67           │ 0.52        │ 0.01       │     0.76 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ DecisionTree       │ 0.90     │ 0.84        │ 0.01       │ 0.59           │ 0.46        │ 0.01       │     0.75 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagged KNN         │ 0.81     │ 0.48        │ 0.01       │ 0.62           │ 0.47        │ 0.01       │     0.72 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ KNN                │ 0.80     │ 0.54        │ 0.00       │ 0.61           │ 0.50        │ 0.01       │     0.71 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ LogisticRegression │ 0.75     │ 0.59        │ 0.01       │ 0.59           │ 0.28        │ 0.00       │     0.67 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ AdaBoost           │ 0.71     │ 0.44        │ 0.05       │ 0.58           │ 0.28        │ 0.00       │     0.65 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ SVM                │ 0.61     │ 0.32        │ 0.01    